##### Install Libraries

In [218]:
# -*- coding: utf-8 -*-
from dreal import *
from Functions import *
import torch 
import torch.nn.functional as F
import numpy as np
import timeit 
import matplotlib.pyplot as plt

## Learning Lyapunov function for CRW Forward Flight

## Neural network model
Building NN with random parameters for Lyapunov function and initializing parameters of NN controller to LQR solution

LQR solution is obtained by minimizing the cost function J = ∫(xᵀQx + uᵀRu)dt, where Q is 2×2 identity matrix and R is 1×1 identity matrix

In [219]:
class Net(torch.nn.Module):
    
    def __init__(self,n_input,n_hidden,n_output,lqr):
        super(Net, self).__init__()
        torch.manual_seed(2)
        self.layer1 = torch.nn.Linear(n_input, n_hidden)
        self.layer2 = torch.nn.Linear(n_hidden,n_output)
        self.control = torch.nn.Linear(n_input,1,bias=False)
        self.control.weight = torch.nn.Parameter(lqr)

    def forward(self,x):
        sigmoid = torch.nn.Tanh()
        h_1 = sigmoid(self.layer1(x))
        out = sigmoid(self.layer2(h_1))
        u = self.control(x)
        return out,u

## CRW Forward Flight Dynamics

In [220]:
def f_value(x,u):
    #Dynamics
    y = []
    g = 9.81 # Gravity
    cl = 0.4 # Standard cl value
    cd = 0.05 # Standard cd value
    p_air = 1.225 # in kg/m^3
    m = 13.5 # in kg
    kn = 500 # in N-m
    A_ref = 0.5 # in m^2
    
    for r in range(0,len(x)): 
        theta_dot = x[r][6]*np.cos(x[r][3])-x[r][7]*np.sin(x[r][3])
        gamma_dot = x[r][5]-g*x[r][2]*x[r][6]*np.cos(x[r][3])+g*x[r][2]*x[r][7]*np.cos(x[r][3])
        psi_dot = -1*(np.sin(x[r][3])/np.cos(x[r][2]))*x[r][6]-(np.cos(x[r][3])/np.cos(x[r][2]))*x[r][7]
        f = [-x[r][1]*x[r][6]-cd*p_air*A_ref*(x[r][0]**2+x[r][1]**2)/(2*m)+u[r][1]/m-kn*u[r][0]*np.sin(u[r][2])/m,
             x[r][0]*x[r][6]+cl*p_air*A_ref*(x[r][0]**2+x[r][1]**2)/(2*m)+kn*u[r][0]*np.sin(u[r][2])/m,
             x[r][6]*np.cos(x[r][3])-x[r][7]*np.sin(x[r][3]),
             x[r][5]-g*x[r][2]*x[r][6]*np.cos(x[r][3])+g*x[r][2]*x[r][7]*np.cos(x[r][3]),
             -1*(np.sin(x[r][3])/np.cos(x[r][2]))*x[r][6]-(np.cos(x[r][3])/np.cos(x[r][2]))*x[r][7],
             u[r][4]+np.sin(x[r][2])*u[r][5]+np.cos(x[r][2])*psi_dot*theta_dot,
             np.sin(x[r][2])*np.sin(x[r][7])*psi_dot*theta_dot-np.cos(x[r][2])*np.cos(x[r][7])*psi_dot*gamma_dot-np.cos(x[r][2])*np.sin(x[r][7])*u[r][5]-np.sin(x[r][7])*theta_dot*gamma_dot+np.cos(x[r][7])*u[r][3],
             np.sin(x[r][2])*np.cos(x[r][7])*psi_dot*theta_dot+np.cos(x[r][2])*np.sin(x[r][7])*psi_dot*gamma_dot-np.cos(x[r][2])*np.cos(x[r][7])*u[r][5]-np.cos(x[r][7])*theta_dot*gamma_dot-np.sin(x[r][7])*u[r][3]]
             
        y.append(f)
    #print(len(y))
    y = torch.tensor(y)
    #y[:,1] = y[:,1] + (u[:,0]/(m*L**2))
    return y

## Options

In [221]:
'''
For learning 
'''
N = 500             # sample size
D_in = 8            # input dimension
H1 = 6              # hidden dimension
D_out = 6           # output dimension
torch.manual_seed(10)  
x = torch.Tensor(N, D_in).uniform_(-6, 6).float()
x_0 = torch.zeros([1, 8])

'''
For verifying 
'''
x1 = Variable("x1")
x2 = Variable("x2")
x3 = Variable("x3")
x4 = Variable("x4")
x5 = Variable("x5")
x6 = Variable("x6")
x7 = Variable("x7")
x8 = Variable("x8")

#u1 = Variable("u1")
#u2 = Variable("u2")
#u3 = Variable("u3")
#u4 = Variable("u4")
#u5 = Variable("u5")
#u6 = Variable("u6")

vars_ = [x1,x2,x3,x4,x5,x6,x7,x8]
g = 9.81 # Gravity
cl = 0.4 # Standard cl value
cd = 0.05 # Standard cd value
p_air = 1.225 # in kg/m^3
m = 13.5 # in kg
kn = 500 # in N-m
A_ref = 0.5 # in m^2

config = Config()
config.use_polytope_in_forall = True
config.use_local_optimization = True
config.precision = 1e-2
epsilon = 0
# Checking candidate V within a ball around the origin (ball_lb ≤ sqrt(∑xᵢ²) ≤ ball_ub)
ball_lb = 0.5
ball_ub = 6

## Learning and Falsification

In [394]:
out_iters = 0
valid = False
while out_iters < 2 and not valid: 
    start = timeit.default_timer()
    lqr = torch.tensor([[0, 0, 0, 0, 0, 0, 0, 0], [3.7825, 3.4403, 0, 0, 0, 0, 0, 0], [0.0033, -0.0158, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0.5545, 0, 0.8322, 0.5545], [0, 0, 0, 1, 0, 1.7321, 0, 0],[0, 0, 0, 0, 0.8322, 0, -0.5545, -4.4532]])    # lqr solution
    model = Net(D_in,H1,D_out,lqr)
    L = []
    i = 0 
    t = 0
    max_iters = 2000
    learning_rate = 0.01
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    while i < max_iters and not valid: 
        V_candidate, u = model(x.float())
        X0,u0 = model(x_0.float())
        f = f_value(x.detach().numpy(),u.detach().numpy())
        Circle_Tuning = Tune(x.float())
        # Compute lie derivative of V : L_V = ∑∂V/∂xᵢ*fᵢ
        L_V = torch.diagonal(torch.mm(torch.mm(torch.mm(dtanh(V_candidate.float()),model.layer2.weight)\
                            *dtanh(torch.tanh(torch.mm(x.float(),model.layer1.weight.t())+model.layer1.bias)),model.layer1.weight),f.t().float()),0)
        Lyapunov_risk = torch.zeros(1,6)
        # With tuning term 
        for m in range(0, 5):
            Lyapunov_risk[0,m] = (F.relu(-V_candidate[:,m])+ 1.5*F.relu(L_V+0.5)).mean()\
                    +2.2*((Circle_Tuning-6*V_candidate[:,m]).pow(2)).mean()+(X0[0,m]).pow(2)
         
        # Without tuning term
#         Lyapunov_risk = (F.relu(-V_candidate)+ 1.5*F.relu(L_V+0.5)).mean()+ 1.2*(X0).pow(2)
        
        
        #print(i, "Lyapunov Risk=",Lyapunov_risk.item()) 
        L.append(Lyapunov_risk[0,0])
        optimizer.zero_grad()
        Lyapunov_risk[0,0].backward()
        optimizer.step() 

        w1 = model.layer1.weight.data.numpy()
        w2 = model.layer2.weight.data.numpy()
        b1 = model.layer1.bias.data.numpy()
        b2 = model.layer2.bias.data.numpy()
        q1 = model.control.weight.data.numpy()[0][:]
        q2 = model.control.weight.data.numpy()[1][:]
        q3 = model.control.weight.data.numpy()[2][:]
        q4 = model.control.weight.data.numpy()[3][:]
        q5 = model.control.weight.data.numpy()[4][:]
        q6 = model.control.weight.data.numpy()[5][:]

        # Falsification
        if i % 10 == 0:
            u1_NN = (q1.item(0)*x1 + q1.item(1)*x2 + q1.item(2)*x3 + q1.item(3)*x4 + q1.item(4)*x5 + q1.item(5)*x6 + q1.item(6)*x7 + q1.item(7)*x8)
            u2_NN = (q2.item(0)*x1 + q2.item(1)*x2 + q2.item(2)*x3 + q2.item(3)*x4 + q2.item(4)*x5 + q2.item(5)*x6 + q2.item(6)*x7 + q2.item(7)*x8)
            u3_NN = (q3.item(0)*x1 + q3.item(1)*x2 + q3.item(2)*x3 + q3.item(3)*x4 + q3.item(4)*x5 + q3.item(5)*x6 + q3.item(6)*x7 + q3.item(7)*x8)
            u4_NN = (q4.item(0)*x1 + q4.item(1)*x2 + q4.item(2)*x3 + q4.item(3)*x4 + q4.item(4)*x5 + q4.item(5)*x6 + q4.item(6)*x7 + q4.item(7)*x8)
            u5_NN = (q5.item(0)*x1 + q5.item(1)*x2 + q5.item(2)*x3 + q5.item(3)*x4 + q5.item(4)*x5 + q5.item(5)*x6 + q5.item(6)*x7 + q5.item(7)*x8)
            u6_NN = (q6.item(0)*x1 + q6.item(1)*x2 + q6.item(2)*x3 + q6.item(3)*x4 + q6.item(4)*x5 + q6.item(5)*x6 + q6.item(6)*x7 + q6.item(7)*x8)
            
            theta_dot = x7*cos(x4)-x8*sin(x4)
            gamma_dot = x6-g*x3*x7*cos(x4)+g*x3*x8*cos(x4)
            psi_dot = -1*(sin(x4)/cos(x3))*x7-(cos(x4)/cos(x3))*x8
        
            f = [-x2*x7-cd*p_air*A_ref*(x1**2+x2**2)/(2*m)+u2_NN/m-kn*u1_NN*sin(u3_NN)/m,
             x1*x7+cl*p_air*A_ref*(x1**2+x2**2)/(2*m)+kn*u1_NN*sin(u3_NN)/m,
             x7*cos(x4)-x8*sin(x4),
             x6-g*x3*x7*cos(x4)+g*x3*x8*cos(x4),
             -1*(sin(x4)/cos(x3))*x7-(cos(x4)/cos(x3))*x8,
             u5_NN+sin(x3)*u6_NN+cos(x3)*psi_dot*theta_dot,
             sin(x3)*sin(x8)*psi_dot*theta_dot-cos(x3)*cos(x8)*psi_dot*gamma_dot-cos(x3)*sin(x8)*u6_NN-sin(x8)*theta_dot*gamma_dot+cos(x8)*u4_NN,
             sin(x3)*cos(x8)*psi_dot*theta_dot+cos(x3)*sin(x8)*psi_dot*gamma_dot-cos(x3)*cos(x8)*u6_NN-cos(x8)*theta_dot*gamma_dot-sin(x8)*u4_NN]
             

            # Candidate V
            z1 = np.dot(vars_,w1.T)+b1

            a1 = []
            for j in range(0,len(z1)):
                a1.append(tanh(z1[j]))
            z2 = np.dot(a1,w2.T)+b2
            V_learn = tanh(z2.item(0))

            print('===========Verifying==========')        
            start_ = timeit.default_timer() 
            result= CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config,epsilon)
            stop_ = timeit.default_timer() 

            if (result): 
                print("Not a Lyapunov function. Found counterexample: ")
                print(result)
                x = AddCounterexamples(x,result,10)
            else:  
                valid = True
                print("Satisfy conditions!!")
                print(V_learn, " is a Lyapunov function.")
            t += (stop_ - start_)
            print('==============================') 
        i += 1

    stop = timeit.default_timer()


    np.savetxt("w1.txt", model.layer1.weight.data, fmt="%s")
    np.savetxt("w2.txt", model.layer2.weight.data, fmt="%s")
    np.savetxt("b1.txt", model.layer1.bias.data, fmt="%s")
    np.savetxt("b2.txt", model.layer2.bias.data, fmt="%s")
    np.savetxt("q.txt", model.control.weight.data, fmt="%s")

    print('\n')
    print("Total time: ", stop - start)
    print("Verified time: ", t)
    
    out_iters+=1

===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-3.0002137683805636, -3.0000000000000004]
x2 : [2.5980762113533165, 2.5983230468862613]
x3 : [-2.253878806900997, -2.2535942783204375]
x4 : [2.422044686410926, 2.4223094597824826]
x5 : [-2.1998380946847687, -2.1995465764411168]
x6 : [1.5347422852417454, 1.5351600221085508]
x7 : [-1.3456535339355469, -1.3453445434570312]
x8 : [0.5480317008323401, 0.54829929443626213]
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-3.0221943877325126, -3.0121943877325128]
x2 : [2.6124417450395088, 2.6224417450395086]
x3 : [-2.2658422345554508, -2.255842234555451]
x4 : [1.9556182476151869, 1.9656182476151867]
x5 : [-1.7058206852572499, -1.6958206852572502]
x6 : [1.4710174249721506, 1.4810174249721504]
x7 : [-1.2865002018203837, -1.2765002018203839]
x8 : [1.1081893923156909, 1.1181893923156907]
===========Verifying==========
Not a Lyapunov function. Found counterexample: 
x1 : [-3.0393

RuntimeError: KeyboardInterrupt(SIGINT) Detected.

## Learning Lyapunov function for CRW in Hover Flight

In [ ]:
epsilon = -0.00001
start_ = timeit.default_timer() 
result = CheckLyapunov(vars_, f, V_learn, ball_lb, ball_ub, config, epsilon)
stop_ = timeit.default_timer() 

if (result): 
    print("Not a Lyapunov function. Found counterexample: ")
else:  
    print("Satisfy conditions with epsilon= ",epsilon)
    print(V_learn, " is a Lyapunov function.")
t += (stop_ - start_)